In [2]:
# Import native modules
import os
import json
import random
from datetime import datetime                    # Use to record time

# Import Keras functions
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense, Activation, concatenate,\
                GlobalAveragePooling2D, Conv2D, Concatenate, AveragePooling2D, BatchNormalization
from keras.utils.np_utils import to_categorical
from keras import backend as backK

# Import matrix and plotting
import numpy as np
import matplotlib
matplotlib.use('Agg')                           # Stops from plotting to screen
import matplotlib.pyplot as plt

# Import OpenCV
import cv2

# Import custom dataset class
from dataset import Dataset

Using TensorFlow backend.


In [3]:
DATASET_NAME = 'plankton_resized'
EPOCHS = 100
BATCH_SIZE = 50
NUM_TRAIN,NUM_VAL,NUM_TEST = 45,10,45

IMAGE_WIDTH,IMAGE_HEIGHT,NUM_CHANNELS = 299,299,3


ID = "{}_{}_{}_{}_{}_{}_{}".format("+2INCEPTION-AVG-2048relu_EXx2",DATASET_NAME,
                                EPOCHS,BATCH_SIZE,NUM_TRAIN,NUM_VAL,NUM_TEST)


# Load dataset
cal = Dataset(DATASET_NAME,IMAGE_HEIGHT,IMAGE_WIDTH,resized=True)
cal.read_data()
cal.train_val_test_split(NUM_TRAIN,NUM_VAL,NUM_TEST)
num_classes = cal.num_classes

Loaded data from plankton_resized
Split 30336 data into 13651 training, 3033 validation, and 13651 testing data.


In [4]:
def logger(message):
    """Logs any message into a file"""
    with open('./models/stats.txt', 'a+') as f:
        print >>f, message
        print(message)

def plot(datas, title, xlabel, ylabel, file_name):
    """Plots the data"""
    plt.figure()
    for key,value in datas.iteritems():
        plt.plot(value, label=key)
    plt.legend(loc='upper right')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    plots_dir = 'plots'
    file_name + '.png'
    plot_path = os.path.join(plots_dir,file_name)    
    plt.savefig(plot_path)

In [5]:
#Taken from inception_v3.py in the keras repository
def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), name=None):
    """Utility function to apply conv + BN (Batch Normalization).
    # Arguments
        x: input tensor.
        filters: filters in `Conv2D`.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv2D`.
        strides: strides in `Conv2D`.
        name: name of the ops; will become `name + '_conv'`
            for the convolution and `name + '_bn'` for the
            batch norm layer.
    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    if backK.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(filters, (num_row, num_col), strides=strides, padding=padding, use_bias=False, name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def load_model():
    """Returns a pretrained model"""
    
    # Loads base model
    base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS))
    print("Model weights loaded.")
    
    # Typical Output
    x = base_model.output
    
    # For evaluating pre-inception layers
    #x = base_model.layers[-32].output  # "-1"st layer
    #x = base_model.layers[-63].output  # "-2"nd layer

    # Inception modules
    inception_count = 2          # number of inception layers to add
    for i in range(inception_count):
        branch1x1 = conv2d_bn(x, 320, 1, 1)

        branch3x3 = conv2d_bn(x, 384, 1, 1)
        branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
        branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
        branch3x3 = concatenate([branch3x3_1, branch3x3_2], axis=3, name='mixed11_'+str(i)+'_added')

        branch3x3dbl = conv2d_bn(x, 448, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
        branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
        branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
        branch3x3dbl = concatenate([branch3x3dbl_1, branch3x3dbl_2], axis=3)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = concatenate([branch1x1, branch3x3, branch3x3dbl, branch_pool], axis=3, name='mixed'+str(12+i)+'_added')
        
    # Add layers (the typical ones)
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dropout(0.2)(x)
    x = Dense(2048,activation='relu')(x)

    # Final fully connected layer to work with our data
    predictions = Dense(num_classes,activation='softmax')(x)

    # Build a final model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    print("Model structure")
    model.summary()
    
    # Compile model
    model.compile(optimizers.SGD(lr=1e-4,momentum=0.9),
                'categorical_crossentropy', metrics=['accuracy'])
    print("Model compiled")

    return model

In [ ]:
description = """Inception V3 - +2 inception + 2048 dense (extended x2) output"""
logger("-----------------------------------------------------------------")
logger(ID)
logger(description)

# Make model
model = load_model()
print("Model created")

# Start time
start_time = datetime.now()

# Load the training data
X_train, Y_train = cal.load_training()

# Print time loading training
elapsed_time = datetime.now() - start_time
logger("Elapsed Time: Loading training")
logger(elapsed_time)   

# Start time
start_time = datetime.now()

# Load the validation data
X_val, Y_val = cal.load_validation()

# Print time loading validation
elapsed_time = datetime.now() - start_time
logger("Elapsed Time: Loading validation")
logger(elapsed_time)  

# Start time
start_time = datetime.now()

# Train model and store stats in history
history = model.fit(x=X_train,y=Y_train,batch_size=BATCH_SIZE,
                    epochs=EPOCHS,validation_data=(X_val,Y_val))

history = history.history

# Print total time
elapsed_time = datetime.now() - start_time
logger("Elapsed Time: Training")
logger(elapsed_time)   

# Save model weights
model_dir = 'models'

model_name = '{}'.format(ID)
model_name += '.h5'
model_path = os.path.join(model_dir,model_name)

model.save(model_path)

logger(history)
print("Model weights saved")

accuracy = {'Training': history['acc'],
            'Validation': history['val_acc']}

loss = {'Training': history['loss'],
        'Validation': history['val_loss']}

# Plot training vs validation accuracy  
plot(accuracy, "Accuracy: Training vs Validation",
        'Epochs', 'Accuracy', '{}_accuracy_train_val'.format(ID))

# Plot training vs validation loss  
plot(loss, "Loss: Training vs Validation",
        'Epochs', 'Loss', '{}_loss_train_val'.format(ID))

# Start time
start_time = datetime.now()

# Loading testing data
X_test, Y_test = cal.load_testing()

# Print time loading testing
elapsed_time = datetime.now() - start_time
logger("Elapsed Time: Loading testing")
logger(elapsed_time)  

# Start time
start_time = datetime.now()

# Test model
metrics = model.evaluate(x=X_test,y=Y_test, batch_size=BATCH_SIZE)

# Print time testing
elapsed_time = datetime.now() - start_time
logger("Elapsed Time: Testing")
logger(elapsed_time)  

logger(metrics)
logger(model.metrics_names)

-----------------------------------------------------------------
+2INCEPTION-AVG-2048relu_EXx2_plankton_resized_100_50_45_10_45
Inception V3 - +2 inception + 2048 dense (extended x2) output
Model weights loaded.
Model structure
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 149, 149, 32)  96          conv2d_1[0][0]                   
________________________________________________________________

Loaded training data 13651/13651   
Elapsed Time: Loading training
0:00:46.712620
Loading validation data...
Loaded validation data033/3033   
Elapsed Time: Loading validation
0:00:10.315346
Train on 13651 samples, validate on 3033 samples
Epoch 1/100
13651/13651 [==============================] - 193s - loss: 4.4053 - acc: 0.0828 - val_loss: 4.3134 - val_acc: 0.0781
Epoch 2/100
13651/13651 [==============================] - 184s - loss: 3.8376 - acc: 0.1802 - val_loss: 3.7653 - val_acc: 0.1929
Epoch 3/100
13651/13651 [==============================] - 184s - loss: 3.4675 - acc: 0.2575 - val_loss: 3.4035 - val_acc: 0.2773
Epoch 4/100
13651/13651 [==============================] - 184s - loss: 3.1490 - acc: 0.3296 - val_loss: 2.9619 - val_acc: 0.3798
Epoch 5/100
13651/13651 [==============================] - 186s - loss: 2.8556 - acc: 0.3948 - val_loss: 2.7806 - val_acc: 0.4224
Epoch 6/100
13651/13651 [==============================] - 185s - loss: 2.6265 - acc: 0.4356 - val_loss: 2.521